In [1]:
# 런타임 초기화 후 제일 먼저
import gc
gc.collect()

123

In [2]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from functools import reduce

In [4]:
train_6 = pd.read_csv('train_5.잔액정보_98이상제거.csv')
segment = pd.read_csv("Segment_병합.csv")

In [5]:
# 1️⃣ concat으로 segment 컬럼 붙이기
train_full = pd.concat([train_6, segment['Segment']], axis=1)

In [6]:
train_full

,기준년월,ID,잔액_일시불_B0M,잔액_할부_B0M,잔액_현금서비스_B0M,잔액_리볼빙일시불이월_B0M,잔액_리볼빙CA이월_B0M,잔액_카드론_B0M,월중평잔_일시불_B0M,월중평잔_할부_B0M,...,평잔_일시불_6M,평잔_일시불_해외_6M,평잔_RV일시불_6M,평잔_RV일시불_해외_6M,평잔_할부_6M,평잔_할부_해외_6M,평잔_CA_6M,평잔_CA_해외_6M,평잔_카드론_6M,Segment
0,201807,TRAIN_000000,998,962,22971,0,0,0,1084,547,...,2440,0,0,0,572,0,17008,0,0,D
1,201807,TRAIN_000001,2565,2390,0,0,0,0,4090,2553,...,2677,0,2830,0,2736,0,0,0,0,E
2,201807,TRAIN_000002,5312,5113,21531,6795,0,0,5006,8778,...,9118,0,8870,0,4429,0,43351,0,0,C
3,201807,TRAIN_000003,730,5025,26284,0,0,0,487,5607,...,884,0,0,0,5097,0,30697,0,0,D
4,201807,TRAIN_000004,0,0,0,0,0,0,0,0,...,21,0,0,0,0,0,0,0,0,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2399995,201812,TRAIN_399995,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E
2399996,201812,TRAIN_399996,3351,0,0,0,0,27337,4412,0,...,12524,0,0,0,0,0,0,0,23031,D
2399997,201812,TRAIN_399997,2524,2960,0,0,0,0,2694,3374,...,3241,0,0,0,3995,0,0,0,0,C
2399998,201812,TRAIN_399998,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,E


In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [8]:
target = train_full['Segment']
X = train_full.drop(columns=['ID', 'Segment'])

In [9]:
X_encoded = pd.get_dummies(X, drop_first=True)

In [10]:
from sklearn.preprocessing import LabelEncoder

# 문자열 target → 숫자형으로 변환
le = LabelEncoder()
target_encoded = le.fit_transform(target)

print("\n✅ 타겟 라벨 인코딩 완료")
print(pd.Series(target_encoded).value_counts())


✅ 타겟 라벨 인코딩 완료
4    1922052
3     349242
2     127590
0        972
1        144
Name: count, dtype: int64


In [11]:
corrs = X_encoded.corrwith(pd.Series(target_encoded))
corrs_sorted = corrs.abs().sort_values(ascending=False)

print("\n✅ 상관계수 (절대값 내림차순):")
print(corrs_sorted)


✅ 상관계수 (절대값 내림차순):
평잔_일시불_6M         4.479670e-01
월중평잔_일시불_B0M      4.446213e-01
월중평잔_일시불          4.415268e-01
평잔_일시불_3M         4.345136e-01
잔액_일시불_B0M        4.173537e-01
                      ...     
연체잔액_현금서비스_B0M    2.996266e-03
잔액_리볼빙CA이월_B0M    1.597595e-03
연체잔액_RV일시불_B0M    5.534963e-04
연체잔액_카드론_B0M      2.469589e-04
기준년월              1.939854e-16
Length: 72, dtype: float64


In [12]:
corrs_sorted = corrs.sort_values(ascending=False)
print(corrs_sorted.round(3))

기준년월             -0.000
연체잔액_카드론_B0M     -0.000
연체잔액_RV일시불_B0M   -0.001
잔액_리볼빙CA이월_B0M   -0.002
연체잔액_현금서비스_B0M   -0.003
                  ...  
잔액_일시불_B0M       -0.417
평잔_일시불_3M        -0.435
월중평잔_일시불         -0.442
월중평잔_일시불_B0M     -0.445
평잔_일시불_6M        -0.448
Length: 72, dtype: float64


In [13]:
corrs_sorted = corrs.reindex(corrs.abs().sort_values(ascending=False).index)
print(corrs_sorted.round(3))


평잔_일시불_6M        -0.448
월중평잔_일시불_B0M     -0.445
월중평잔_일시불         -0.442
평잔_일시불_3M        -0.435
잔액_일시불_B0M       -0.417
                  ...  
연체잔액_현금서비스_B0M   -0.003
잔액_리볼빙CA이월_B0M   -0.002
연체잔액_RV일시불_B0M   -0.001
연체잔액_카드론_B0M     -0.000
기준년월             -0.000
Length: 72, dtype: float64


In [14]:
import pandas as pd

# 모든 행 출력
pd.set_option('display.max_rows', None)

# 절댓값 기준 정렬 + 부호 유지
corrs_sorted = corrs.reindex(corrs.abs().sort_values(ascending=False).index)

# 소수점 3자리까지 출력
print(corrs_sorted.round(3))


평잔_일시불_6M         -0.448
월중평잔_일시불_B0M      -0.445
월중평잔_일시불          -0.442
평잔_일시불_3M         -0.435
잔액_일시불_B0M        -0.417
잔액_일시불_B1M        -0.374
잔액_일시불_B2M        -0.373
평잔_일시불_해외_6M      -0.313
평잔_일시불_해외_3M      -0.293
월중평잔              -0.287
평잔_6M             -0.281
평잔_CA_6M          -0.274
평잔_3M             -0.268
평잔_CA_3M          -0.268
월중평잔_CA           -0.266
잔액_현금서비스_B2M      -0.265
잔액_현금서비스_B0M      -0.265
잔액_현금서비스_B1M      -0.265
잔액_할부_B0M         -0.264
연체원금_최근           -0.264
연체일수_최근           -0.264
월중평잔_CA_B0M       -0.264
평잔_할부_6M          -0.250
잔액_할부_무이자_B0M     -0.250
월중평잔_할부_B0M       -0.235
평잔_할부_3M          -0.233
월중평잔_할부           -0.232
잔액_할부_B1M         -0.229
잔액_할부_B2M         -0.228
잔액_할부_유이자_B0M     -0.146
평잔_RV일시불_해외_6M    -0.122
평잔_RV일시불_6M       -0.120
월중평잔_RV일시불        -0.120
평잔_RV일시불_해외_3M    -0.118
평잔_RV일시불_3M       -0.117
RV_최대잔액_R12M      -0.076
RV_최대잔액_R6M       -0.073
RV_최대잔액_R3M       -0.071
잔액_리볼빙일시불이월_B0M   -0.070
RV_평균잔액_R3M       -0.067
